# The Genesis process

Creating a new game requires the creation of a system and planets, factions and people.

This notebook tests that process as well as documents it. This notebook tests several attributes of the generation process to ensure that the code is sound. 

In [1]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")

%load_ext lab_black

In [2]:
from app import creators
from app import objects

In [3]:
from app.creators import homeworld
from app.creators import universe

An example `data` set for testing. This is the user form that the user submits when creating a new game. 

You can get this from the template at `app\templates\app\creation\genesis.js`


In [4]:
data = {
    "label": "form",
    "name": "worldgenform",
    "objid": "0000000000001",
    "owner": "user.username",
    "username": "user.username",
    "accountid": "0000000000001",
    "conformity": 0.5,
    "constitution": 0.5,
    "literacy": 0.5,
    "aggression": 0.5,
    "num_planets": 4,
    "num_moons": 10,
    "starting_pop": 7,
    "organics": 0.5,
    "minerals": 0.5,
}

You'll also need to configuration files.

In [5]:
conf = creators.universe.configurations.get_configurations()

Each object as an `__init__` function that creates it, however not all data is spawed on init. 

## The Solar System and Planets

In [6]:
system = universe.celestials.System(data)
system

<system: ordered; 6411502708068; Maitilher>

In [7]:
system.get_data()

{'objid': '6411502708068',
 'name': 'Maitilher',
 'label': 'system',
 'class': 'ordered',
 'isHomeSystem': True,
 'glat': 22.212,
 'glon': -17.666,
 'gelat': -3.19}

Each object as an `__init__` function that creates it, and populates it with data available at that time. Celestial objects have dependancies on other objects. 

In [8]:
star = objects.celestials.Star(conf["star_config"], system)
star.get_data()

{'name': 'Yubei',
 'class': 'G',
 'objid': '7065383381300',
 'label': 'star',
 'radius': 106}

Each object also inherits a `get_fundamentals()` method that ensures that needed default values are present. `get_data()` extends that functionality. This ensures that objects can alwasy interact with the graph. 

In [9]:
star.get_fundimentals()

{'name': 'Yubei', 'class': 'G', 'objid': '7065383381300', 'label': 'star'}

Procedurally generated planets chose from a list of potential types in the `conf`. To force a particular kind of outcome, reduce the options in the configuration. 

In [10]:
terrestrial_config = {"terrestrial": conf["planet_config"]["terrestrial"]}
home_planet = objects.celestials.Planet(conf=terrestrial_config, orbiting=star)
home_planet.get_data()

{'name': 'Herdro',
 'class': 'terrestrial',
 'objid': '6068983698180',
 'label': 'planet',
 'radius': 0.49,
 'mass': 0.856,
 'orbitsDistance': 0.661,
 'orbitsId': '7065383381300',
 'orbitsName': 'Yubei',
 'isSupportsLife': False,
 'isPopulated': False}

Creating a group of planets using list comprehension. Note that celestial objects have a custom `__repr__` function that makes them easy to manage.

In [11]:
planets = [
    objects.celestials.Planet(conf=conf["planet_config"], orbiting=star)
    for p in range(int(data["num_planets"]) - 1)
]
planets

[<planet: gas; 0273281284061; Mendinjudon>,
 <planet: ice; 3122430423239; Ko>,
 <planet: terrestrial; 7806131661528; Fenbou>]

In [12]:
moons = [
    objects.celestials.Moon(conf["moon_config"], planets)
    for p in range(int(data["num_moons"]))
]
moons

[<moon: ice; 2603053040818; Pitha>,
 <moon: terrestrial; 6897208941331; Hia>,
 <moon: ice; 1933394281955; Me>,
 <moon: rocky; 0900933555137; Dertal>,
 <moon: ice; 1635443774366; Tsurubat>,
 <moon: ice; 6830196725820; Rezloswoganj>,
 <moon: rocky; 7143685931919; Rahamna>,
 <moon: rocky; 0018876517065; Labal>,
 <moon: rocky; 1526709402449; Franci>,
 <moon: ice; 3309147272094; Sam>]

Getting the nodes and edges of each by calling the `self.orbiting` propperty.

In [13]:
moons[0].orbiting

<planet: terrestrial; 7806131661528; Fenbou>

In [14]:
home_planet.orbiting

<star: G; 7065383381300; Yubei>

Additionally, you can quickly navigate the system by referencing other objects. 

In [15]:
home_planet.orbiting.system

<system: ordered; 6411502708068; Maitilher>

Getting the nodes for the graph. Sandwich all of the items together and get the data using the same generic function.

In [16]:
all_entities = [system] + [star] + moons + planets + [home_planet]
all_nodes = [b.get_data() for b in all_entities]
pd.DataFrame(all_nodes)

,objid,name,label,class,isHomeSystem,glat,glon,gelat,radius,orbitsId,orbitsName,orbitsDistance,mass,isSupportsLife,isPopulated
0,6411502708068,Maitilher,system,ordered,True,22.212,-17.666,-3.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7065383381300,Yubei,star,G,NaN,NaN,NaN,NaN,106.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,2603053040818,Pitha,moon,ice,NaN,NaN,NaN,NaN,0.023282,7806131661528,Fenbou,0.5331,0.031931,False,False
3,6897208941331,Hia,moon,terrestrial,NaN,NaN,NaN,NaN,0.125233,7806131661528,Fenbou,0.5331,0.017473,False,False
4,1933394281955,Me,moon,ice,NaN,NaN,NaN,NaN,0.018832,7806131661528,Fenbou,0.5770,0.018010,False,False
5,0900933555137,Dertal,moon,rocky,NaN,NaN,NaN,NaN,0.647380,0273281284061,Mendinjudon,4.0001,0.000396,False,False
6,1635443774366,Tsurubat,moon,ice,NaN,NaN,NaN,NaN,0.153850,0273281284061,Mendinjudon,4.1140,0.011020,False,False
7,6830196725820,Rezloswoganj,moon,ice,NaN,NaN,NaN,NaN,0.136532,3122430423239,Ko,3.8820,0.032780,False,False
8,7143685931919,Rahamna,moon,rocky,NaN,NaN,NaN,NaN,0.117140,7806131661528,Fenbou,0.5550,0.000076,False,False
9,0018876517065,Labal,moon,rocky,NaN,NaN,NaN,NaN,0.143461,7806131661528,Fenbou,0.5331,0.000172,False,False


Getting the edge values to update the graph: 

In [17]:
orbiting_bodies = [home_planet] + planets + moons

orbiting_edges = [i.get_orbits_edge() for i in orbiting_bodies]

pd.DataFrame(orbiting_edges)

,node1,node2,label,orbit_distance
0,6068983698180,7065383381300,orbits,0.6610
1,0273281284061,7065383381300,orbits,6.9540
2,3122430423239,7065383381300,orbits,19.3850
3,7806131661528,7065383381300,orbits,0.7480
4,2603053040818,7806131661528,orbits,0.0001
5,6897208941331,7806131661528,orbits,0.0001
6,1933394281955,7806131661528,orbits,0.0440
7,0900933555137,0273281284061,orbits,0.0001
8,1635443774366,0273281284061,orbits,0.1140
9,6830196725820,3122430423239,orbits,0.0790


In [18]:
pd.DataFrame(orbiting_edges).groupby(["node1", "node2"]).count()["label"].mean() == 1

True

In [19]:
pd.DataFrame(orbiting_edges).groupby(["node2"]).count()

,node1,label,orbit_distance
node2,,,
0273281284061,2,2,2
3122430423239,2,2,2
7065383381300,4,4,4
7806131661528,6,6,6


Check the orbiting logic

In [20]:
[
    f"{i.type} {i.label} orbits a {i.orbiting.type} {i.orbiting.label}"
    for i in orbiting_bodies
]

['terrestrial planet orbits a G star',
 'gas planet orbits a G star',
 'ice planet orbits a G star',
 'terrestrial planet orbits a G star',
 'ice moon orbits a terrestrial planet',
 'terrestrial moon orbits a terrestrial planet',
 'ice moon orbits a terrestrial planet',
 'rocky moon orbits a gas planet',
 'ice moon orbits a gas planet',
 'ice moon orbits a ice planet',
 'rocky moon orbits a terrestrial planet',
 'rocky moon orbits a terrestrial planet',
 'rocky moon orbits a terrestrial planet',
 'ice moon orbits a ice planet']

In [21]:
system_bodies = orbiting_bodies + [star]

system_edges = [i.get_in_system_edge() for i in system_bodies]

pd.DataFrame(system_edges)

,node1,node2,label
0,6068983698180,6411502708068,isIn
1,0273281284061,6411502708068,isIn
2,3122430423239,6411502708068,isIn
3,7806131661528,6411502708068,isIn
4,2603053040818,6411502708068,isIn
5,6897208941331,6411502708068,isIn
6,1933394281955,6411502708068,isIn
7,0900933555137,6411502708068,isIn
8,1635443774366,6411502708068,isIn
9,6830196725820,6411502708068,isIn


### Scanning the homeworld
the homeworld already has some resources known. `scan_body()` is inherited by the base object. So everyone should be able to do it. 

In [22]:
home_planet.scan_body()
home_planet.resources

[<resource: 5526583198661; Organic>,
 <resource: 2060720595831; Common Minerals>,
 <resource: 3634573289058; Rare Minerals>,
 <resource: 9008014284135; Water>]

In [23]:
pd.DataFrame([i.get_data() for i in home_planet.resources])

,name,objid,label,volume,max_volume,description,replenish_rate
0,Organic,5526583198661,resource,1163,1163,bilogical material that can be consumed by pops,10.0
1,Common Minerals,2060720595831,resource,116,116,Iron and other common material used in constru...,NaN
2,Rare Minerals,3634573289058,resource,48,48,"lithium, silver and other rare minerals used i...",NaN
3,Water,9008014284135,resource,12621,12621,"H2O ready to be consumed, either frozen or in ...",NaN


In [24]:
pd.DataFrame([i.get_location_edge() for i in home_planet.resources])

,node1,node2,label
0,6068983698180,5526583198661,has
1,6068983698180,2060720595831,has
2,6068983698180,3634573289058,has
3,6068983698180,9008014284135,has


In [25]:
[
    f"{i.location.type} {i.location.label} has {i.volume} {i.name}"
    for i in home_planet.resources
]

['terrestrial planet has 1163 Organic',
 'terrestrial planet has 116 Common Minerals',
 'terrestrial planet has 48 Rare Minerals',
 'terrestrial planet has 12621 Water']

# The Full Automated Process

Usefull as QA, to ensure that the process will run end-to-end

In [26]:
homesystem_data = universe.build_homeSystem(data, username="notebook")
pd.DataFrame(homesystem_data["nodes"])

,objid,name,label,class,isHomeSystem,glat,glon,gelat,radius,orbitsId,...,accountid,conformity,constitution,literacy,aggression,num_planets,num_moons,starting_pop,organics,minerals
0,8293654076292,Marplatadtnan,system,ordered,True,23.5,-17.098,-3.253,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9848093035029,Norplaeast,star,G,NaN,NaN,NaN,NaN,106.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9746200993442,Heightssindopar,moon,terrestrial,NaN,NaN,NaN,NaN,0.007386,6608770786108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9946280470682,Radaltan,moon,rocky,NaN,NaN,NaN,NaN,0.047079,5811147728894,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4917109242003,New,moon,rocky,NaN,NaN,NaN,NaN,0.001030,6608770786108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,8745701801728,Herganjhalhui,moon,rocky,NaN,NaN,NaN,NaN,0.016125,3316529275778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2170053219496,Mis,moon,rocky,NaN,NaN,NaN,NaN,0.007770,5902578539012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6146025878987,Daker,moon,terrestrial,NaN,NaN,NaN,NaN,0.233305,5811147728894,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5766150125793,Haunue,moon,terrestrial,NaN,NaN,NaN,NaN,0.085096,6608770786108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3058011388076,Ongorrou,moon,rocky,NaN,NaN,NaN,NaN,0.260472,6608770786108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Check** Making sure that there are no duplicate nodes. This dataframe should be empty. 

In [27]:
dup_edges = pd.DataFrame(homesystem_data["nodes"])
dup_edges[dup_edges.duplicated(["objid"])]

,objid,name,label,class,isHomeSystem,glat,glon,gelat,radius,orbitsId,...,accountid,conformity,constitution,literacy,aggression,num_planets,num_moons,starting_pop,organics,minerals


In [28]:
type("True") == bool

False

In [29]:
print(
    f"{len(homesystem_data['edges'])}  total edges for {len(homesystem_data['nodes'])} objects"
)
pd.DataFrame(homesystem_data["edges"]).groupby("label").count()

35  total edges for 21 objects


,node1,node2,orbit_distance
label,,,
createdFrom,1,1,0
has,4,4,0
isIn,15,15,0
orbits,14,14,14
submitted,1,1,0


**Check** Making sure that there are no duplicate edges. This dataframe should be empty. 

In [30]:
dup_edges = pd.DataFrame(homesystem_data["edges"])
dup_edges[dup_edges.duplicated(["node1", "node2"])]

,node1,node2,label,orbit_distance


### Rulse For Ontology:
here is a point to check that the ontology matches this format:
* everything camelCase (starting on lower)
* Don't reference the subject in the edge label (has not hasResource)
* Every node is a Class that can be found in `objects`
* Form and Account are the exception

# The planet Surface

Entirely separate processes, the homeworld makes the population on a planet that already exists. 

In [31]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [32]:
spec = objects.species.Species(data)
spec

<species: None; 9690905184503; Dou>

In [33]:
pops = [objects.population.Pop(spec) for i in range(int(data["starting_pop"]))]
pops_df = pd.DataFrame([p.get_data() for p in pops])
pops_df

,name,objid,label,conformity,literacy,aggression,constitution,health,isInFaction,industry,wealth,factionLoyalty,isIdle
0,unnamed,4880368741174,pop,0.438,0.751,0.635,0.405,0.5,None,0.5200,0.63550,0.337,True
1,unnamed,0829670023932,pop,0.447,0.445,0.394,0.594,0.5,None,0.4940,0.46950,0.322,True
2,unnamed,4870797324390,pop,0.534,0.483,0.321,0.462,0.5,None,0.3915,0.43725,0.493,True
3,unnamed,0912329446429,pop,0.519,0.482,0.496,0.395,0.5,None,0.4455,0.46375,0.505,True
4,unnamed,5648590660696,pop,0.473,0.391,0.433,0.448,0.5,None,0.4405,0.41575,0.478,True
5,unnamed,9797573980812,pop,0.570,0.445,0.421,0.529,0.5,None,0.4750,0.46000,0.415,True
6,unnamed,8526322628235,pop,0.529,0.404,0.583,0.418,0.5,None,0.5005,0.45225,0.541,True


Populations (_pops_) are born with stats that are within normal distribution for it's species. They are not in a faction and don't have names. The faction will help to determine their names. 

In [34]:
n_steps = 6
n_factions = homeworld.get_n_factions(n_steps, float(data["conformity"]))
n_factions

2

Using conformity to get the number of factions. Higher conformity, fewer factions. 

In [35]:
starting_attributes = ["conformity", "literacy", "aggression", "constitution"]

kmeans = KMeans(n_clusters=n_factions).fit(
    pops_df[[c for c in pops_df.columns if c in starting_attributes]]
)
kmeans

KMeans(n_clusters=2)

In [36]:
factions = [objects.population.Faction(i) for i in range(kmeans.n_clusters)]
factions

[<faction: None; 5131113687911; Jehenquimun>,
 <faction: None; 9068674296357; Cor>]

In [37]:
kmeans.labels_

array([1, 0, 0, 0, 0, 0, 0])

In [38]:
for i, n in enumerate(kmeans.labels_):
    pops[i].set_faction(factions[n])
    print(f"pop: {pops[i]} belongs to faction: {factions[n]}")

pop: <pop: pop; 4880368741174; Cor Namve> belongs to faction: <faction: None; 9068674296357; Cor>
pop: <pop: pop; 0829670023932; Jehenquimun Per> belongs to faction: <faction: None; 5131113687911; Jehenquimun>
pop: <pop: pop; 4870797324390; Jehenquimun Cute> belongs to faction: <faction: None; 5131113687911; Jehenquimun>
pop: <pop: pop; 0912329446429; Jehenquimun Der> belongs to faction: <faction: None; 5131113687911; Jehenquimun>
pop: <pop: pop; 5648590660696; Jehenquimun Pen> belongs to faction: <faction: None; 5131113687911; Jehenquimun>
pop: <pop: pop; 9797573980812; Jehenquimun Mel> belongs to faction: <faction: None; 5131113687911; Jehenquimun>
pop: <pop: pop; 8526322628235; Jehenquimun Nigar> belongs to faction: <faction: None; 5131113687911; Jehenquimun>


The name of the population should match the name of the faction. 

In [39]:
pops_df = pd.DataFrame([p.get_data() for p in pops])
pops_df

,name,objid,label,conformity,literacy,aggression,constitution,health,isInFaction,industry,wealth,factionLoyalty,isIdle
0,Cor Namve,4880368741174,pop,0.438,0.751,0.635,0.405,0.5,9068674296357,0.5200,0.63550,0.337,True
1,Jehenquimun Per,0829670023932,pop,0.447,0.445,0.394,0.594,0.5,5131113687911,0.4940,0.46950,0.322,True
2,Jehenquimun Cute,4870797324390,pop,0.534,0.483,0.321,0.462,0.5,5131113687911,0.3915,0.43725,0.493,True
3,Jehenquimun Der,0912329446429,pop,0.519,0.482,0.496,0.395,0.5,5131113687911,0.4455,0.46375,0.505,True
4,Jehenquimun Pen,5648590660696,pop,0.473,0.391,0.433,0.448,0.5,5131113687911,0.4405,0.41575,0.478,True
5,Jehenquimun Mel,9797573980812,pop,0.570,0.445,0.421,0.529,0.5,5131113687911,0.4750,0.46000,0.415,True
6,Jehenquimun Nigar,8526322628235,pop,0.529,0.404,0.583,0.418,0.5,5131113687911,0.5005,0.45225,0.541,True


Just as the pops reference the faction, the faction has a list of it's populations. 

In [40]:
factions[0].pops

[<pop: pop; 0829670023932; Jehenquimun Per>,
 <pop: pop; 4870797324390; Jehenquimun Cute>,
 <pop: pop; 0912329446429; Jehenquimun Der>,
 <pop: pop; 5648590660696; Jehenquimun Pen>,
 <pop: pop; 9797573980812; Jehenquimun Mel>,
 <pop: pop; 8526322628235; Jehenquimun Nigar>]

Creating a PCA to separate the factions by distance. 

In [41]:
if n_factions >= 2:
    # using PCA to set populations on map:

    # PCA Part
    pca = PCA(n_components=2)
    X_r = pca.fit(kmeans.cluster_centers_).transform(kmeans.cluster_centers_)
    for i, f in enumerate(factions):
        f.pca_explained_variance_ratio = pca.explained_variance_ratio_
        f.lat = np.round(X_r[i][0], 3)
        f.long = np.round(X_r[i][1], 3)
else:
    # Only one faction, the lat and long is 0,0
    for i, f in enumerate(factions):
        f.lat = 0
        f.long = 0

In [42]:
pca.explained_variance_ratio_

array([1.00000000e+00, 5.48729495e-32])

In [43]:
factions[0].get_pop_edges([])

[{'node1': '0829670023932', 'node2': '5131113687911', 'label': 'isInFaction'},
 {'node1': '4870797324390', 'node2': '5131113687911', 'label': 'isInFaction'},
 {'node1': '0912329446429', 'node2': '5131113687911', 'label': 'isInFaction'},
 {'node1': '5648590660696', 'node2': '5131113687911', 'label': 'isInFaction'},
 {'node1': '9797573980812', 'node2': '5131113687911', 'label': 'isInFaction'},
 {'node1': '8526322628235', 'node2': '5131113687911', 'label': 'isInFaction'}]

In [44]:
[p.isOfSpecies for p in pops]

[{'node1': '4880368741174', 'node2': '9690905184503', 'label': 'isOfSpecies'},
 {'node1': '0829670023932', 'node2': '9690905184503', 'label': 'isOfSpecies'},
 {'node1': '4870797324390', 'node2': '9690905184503', 'label': 'isOfSpecies'},
 {'node1': '0912329446429', 'node2': '9690905184503', 'label': 'isOfSpecies'},
 {'node1': '5648590660696', 'node2': '9690905184503', 'label': 'isOfSpecies'},
 {'node1': '9797573980812', 'node2': '9690905184503', 'label': 'isOfSpecies'},
 {'node1': '8526322628235', 'node2': '9690905184503', 'label': 'isOfSpecies'}]

Faction's pops should all be placed on a grid. `[0,0]` is the location of the faction. The pops are placed around that faction center.

In [45]:
factions[0].faction_place

[[0, 0], [0, 2], [0, -2], [-1, 1], [-2, 2], [1, 1], [2, 0]]

In [51]:
factions[0].pops

[<pop: pop; 0829670023932; Jehenquimun Per>,
 <pop: pop; 4870797324390; Jehenquimun Cute>,
 <pop: pop; 0912329446429; Jehenquimun Der>,
 <pop: pop; 5648590660696; Jehenquimun Pen>,
 <pop: pop; 9797573980812; Jehenquimun Mel>,
 <pop: pop; 8526322628235; Jehenquimun Nigar>]

In [52]:
len(factions[0].faction_place) - 1 == len(factions[0].pops)

True